In [14]:
import tensorflow as tf
import numpy as np

In [27]:
emb_encoder=a1
print(emb_encoder.shape.as_list())


encoder_outputs=[emb_encoder]
PAD_emb=tf.nn.embedding_lookup(emb,[1])

for enc_layer in range(6):
    with tf.variable_scope("encoder_%d"%enc_layer):
        emb_encoder=tf.pad(emb_encoder,paddings=[[0,0],[1,1],[0,0]],constant_values=PAD_emb)
        print(emb_encoder.shape.as_list())
        encoder_shape=emb_encoder.shape.as_list()

        emb_encoder=tf.reshape(emb_encoder,shape=[encoder_shape[0],encoder_shape[1],encoder_shape[2],1])# batch_size,seq_len,emb_dim,1

        filtersize=[3,10,1,20]
        filter=tf.Variable(initial_value=tf.truncated_normal(filtersize,stddev=0.01),name="filter")
        
        emb_encoder=tf.nn.conv2d(emb_encoder,filter=filter,strides=[1,1,1,1],padding="VALID",dilations=[1,1,1,1])
        emb_encoder=tf.reshape(emb_encoder,shape=[6,-1,10*2])
        A,B=tf.split(emb_encoder,2,axis=2)
        attn=tf.multiply(A,tf.nn.softmax(B))

        print(attn.shape.as_list())

        emb_encoder=attn+encoder_outputs[-1]
        a=tf.nn.softmax(emb_encoder,axis=1)
        print("a",a.shape.as_list())
        encoder_outputs.append(emb_encoder)
        print(emb_encoder.shape.as_list())



[6, 80, 10]
[6, 82, 10]
[6, 80, 10]
a [6, 80, 10]
[6, 80, 10]
[6, 82, 10]
[6, 80, 10]
a [6, 80, 10]
[6, 80, 10]
[6, 82, 10]
[6, 80, 10]
a [6, 80, 10]
[6, 80, 10]
[6, 82, 10]
[6, 80, 10]
a [6, 80, 10]
[6, 80, 10]
[6, 82, 10]


[6, 80, 10]
a [6, 80, 10]
[6, 80, 10]
[6, 82, 10]
[6, 80, 10]
a [6, 80, 10]
[6, 80, 10]


In [6]:
a=np.array(range(48))
a=a.reshape([6,8])
a=tf.constant(a,dtype=tf.float32)
b=tf.constant([1,0,1,1,0,0],dtype=tf.float32,shape=[6,1])
g=tf.constant([[1,0,1,1,0,0],[1,0,1,1,0,0]],dtype=tf.float32,shape=[2,6])

c=tf.multiply(b,a)
init=tf.global_variables_initializer()
sess=tf.Session()
sess.run(init)
d,f=sess.run([c,tf.nn.l2_normalize(g,1)])
print(d)
print(f)

[[  0.   1.   2.   3.   4.   5.   6.   7.]
 [  0.   0.   0.   0.   0.   0.   0.   0.]
 [ 16.  17.  18.  19.  20.  21.  22.  23.]
 [ 24.  25.  26.  27.  28.  29.  30.  31.]
 [  0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.]]
[[ 0.57735026  0.          0.57735026  0.57735026  0.          0.        ]
 [ 0.57735026  0.          0.57735026  0.57735026  0.          0.        ]]


In [10]:
import tensorflow as tf 
a = tf.constant(1)
n = tf.constant(10)

def cond(a, n):
    return  a< n
def body(a,n):
    a = a + 1
    return a,n

a, n = tf.while_loop(cond, body, [a, n])
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    res = sess.run([a,n])
    print(res)


[10, 10]


In [52]:

a=tf.constant([[123,34],[12,45],[56,23],[23,45],[65,13]],dtype=tf.int32)
b=tf.constant([[5],[7],[3],[1],[6]],tf.int32)
tf.global_variables_initializer()
sess=tf.Session()
v=sess.run([a*b])
print(v)


[array([[615, 170],
       [ 84, 315],
       [168,  69],
       [ 23,  45],
       [390,  78]], dtype=int32)]


In [7]:
class A():
    def __init__(self,a,b):
        self.a=a
        self.b=b
    
    def __call__(self, rouge):
        if "a"==rouge:
            return self.a+self.b
        elif "b" ==rouge:
            return self.a*self.b

a=A(3,2)
print(a("a"))


        

5


In [43]:
a=tf.Variable([])
table = tf.TensorArray(tf.int32, 12, clear_after_read=False, element_shape=[])
table=table.write(1,5)
table = tf.reshape(table.stack(), [3, 4])
print(table)
value=table[0,1]
value=tf.reshape(value,shape=[1])
value=tf.cast(value,tf.float32)
print(value)
a=tf.concat([a,value],0)
a=tf.concat([a,value],0)
init=tf.global_variables_initializer()
sess=tf.Session()
sess.run(init)
print(sess.run(a))

Tensor("Reshape_17:0", shape=(3, 4), dtype=int32)
Tensor("Cast_7:0", shape=(1,), dtype=float32)
[5. 5.]


In [50]:
#词汇表中对应的主题词
a=tf.constant([[11,1],[12,2],[13,3],[14,4],[15,5],[16,6]],dtype=tf.int32)
# 词汇表
# b=tf.constant([[1,11,13],[3,16,6],[5,6,2]],dtype=tf.int32)
b=tf.constant([[0,13],[1,16],[0,8]],dtype=tf.int32)

emb=tf.get_variable(name='embedding',shape=[20,5],dtype=tf.float32)
topic_emb=tf.get_variable(name='topics',shape=[10,5],dtype=tf.float32)

In [45]:
a_shape=a.get_shape()[0]
b_shape=b.get_shape()
size=b_shape[0]*b_shape[1]

def look_step(g,index,single,k1):
    t=tf.cond(tf.equal(single,a[k1,0]),
            lambda :(True,k1),
            lambda :(False,-1))
    return tf.logical_or(g,t[0]),tf.maximum(index,t[1]),single,k1+1
    
def _look(single):
    g=False
    k1=0
    index=-1
    g,index,*_=tf.while_loop(
        cond=lambda g,index,single,k1:k1<a_shape,
        body=look_step,
        loop_vars=[g,index,single,k1]
    )
    return g,index

k=tf.constant(1)
def loop_step(k,topic,matrix):
    g,index=_look(topic[k])
    matr=tf.cond(g,
                lambda :tf.gather(topic_emb,[a[index,1]]),
                lambda :tf.gather(emb,[topic[k]]))
    matr=tf.reshape(matr,[1,1,-1])
    matrix=tf.concat([matrix,matr],0)

    return k+1,topic,matrix

g=tf.reshape(b,[-1])
# matrix=tf.Variable([[]])
matrix=tf.nn.embedding_lookup(emb,[0])
matrix=tf.reshape(matrix,[1,1,-1])
_,_,matrix=tf.while_loop(
    cond=lambda k,b,*_:k<size,
    body=loop_step,
    loop_vars=[k,g,matrix],
    shape_invariants=[k.get_shape(),tf.TensorShape([None]),tf.TensorShape([None,1,5])]

)

matrix=tf.reshape(matrix,shape=[-1,b_shape[1],5])
print(matrix)
init=tf.global_variables_initializer()
sess=tf.Session()
sess.run(init)

matrix,topics,emm=sess.run([matrix,topic_emb,emb])
print(np.array(matrix).shape)
print(matrix)

print('------')
print(topics)
print('------')
print(emm)

Tensor("Reshape_21:0", shape=(?, 2, 5), dtype=float32)


(3, 2, 5)
[[[ 0.03026214  0.28801247  0.40880844 -0.43810743  0.21062544]
  [-0.60613453 -0.28839117 -0.56303763  0.296337   -0.00902414]]

 [[-0.3305534   0.01545647  0.0331274  -0.4084127   0.23705521]
  [-0.26574537  0.5975633   0.01666689  0.01336354 -0.39244384]]

 [[ 0.03026214  0.28801247  0.40880844 -0.43810743  0.21062544]
  [-0.3024444  -0.1936149   0.07234898 -0.2342152   0.3469616 ]]]
------
[[ 0.6007559  -0.4593113   0.09159327 -0.478335   -0.09639078]
 [ 0.31148964  0.02132171 -0.08892065  0.5141186   0.58552164]
 [ 0.27487475 -0.47561824  0.16658598 -0.09853393  0.60222894]
 [-0.60613453 -0.28839117 -0.56303763  0.296337   -0.00902414]
 [-0.14825061  0.19814873  0.543827    0.4740948  -0.52177584]
 [ 0.5068485  -0.46798617 -0.38487118 -0.14322299 -0.6246252 ]
 [-0.26574537  0.5975633   0.01666689  0.01336354 -0.39244384]
 [ 0.6091011  -0.27581295  0.5293644  -0.6094516   0.4994821 ]
 [ 0.14774007  0.19275963 -0.33168998 -0.06496447  0.06964922]
 [ 0.41561526  0.29858875 

In [46]:
i = tf.constant(0)
l = tf.Variable([])
array = tf.Variable(tf.random_normal([10]))
def body(i, l):                                               
    temp = tf.gather(array,i)
    l = tf.concat([l, [temp]], 0)
    return i+2, l
def cond(i,l):
   return i < 10
index, list_vals = tf.while_loop(cond, body, [i, l],shape_invariants=[i.get_shape(),
                                                   tf.TensorShape([None])])
sess = tf.Session()
sess.run(tf.global_variables_initializer())
a1,b1=sess.run([list_vals,array])
print(a1)
print(b1)

[-0.12611371  0.08051919  2.6596558  -2.5261319   1.4858145 ]
[-0.12611371 -1.0288202   0.08051919 -0.40016657  2.6596558   0.4868604
 -2.5261319   0.5280155   1.4858145  -0.3094576 ]


In [47]:
gh1=tf.gather(emb,[1])
gh2=tf.gather(emb,[0])
gh=tf.concat([gh2,gh1],axis=0)
print(gh)
a1,b1=sess.run([emb,gh])

Tensor("concat_18:0", shape=(2, 5), dtype=float32)
